# Derivatives

## Overview
Derivatives are financial instruments whose value is **derived** from an underlying asset (stocks, bonds, commodities, currencies, interest rates). They are used for hedging risk, speculation, and arbitrage.

## Types of Derivatives

| Type | Description | Exchange-Traded? | Customizable? |
|------|-------------|------------------|---------------|
| **Forwards** | Customized contract to buy/sell at future date | No (OTC) | Yes |
| **Futures** | Standardized forward contracts | Yes | No |
| **Options** | Right (not obligation) to buy/sell | Both | Varies |
| **Swaps** | Exchange of cash flow streams | No (OTC) | Yes |

## Options Fundamentals

### Call Option
- **Right to BUY** the underlying at the strike price
- Buyer profits when price rises above strike
- Payoff: $\max(S_T - K, 0)$

### Put Option
- **Right to SELL** the underlying at the strike price
- Buyer profits when price falls below strike
- Payoff: $\max(K - S_T, 0)$

### Moneyness
- **In-the-money (ITM)**: Would have value if exercised now
- **At-the-money (ATM)**: Strike ≈ Current price
- **Out-of-the-money (OTM)**: No intrinsic value

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Option Payoff Diagrams
S = np.linspace(50, 150, 200)  # Stock price at expiration
K = 100  # Strike price
premium_call = 5
premium_put = 4

# Payoffs
long_call = np.maximum(S - K, 0) - premium_call
short_call = -np.maximum(S - K, 0) + premium_call
long_put = np.maximum(K - S, 0) - premium_put
short_put = -np.maximum(K - S, 0) + premium_put

fig = make_subplots(rows=2, cols=2, subplot_titles=(
    'Long Call', 'Short Call', 'Long Put', 'Short Put'
))

positions = [(long_call, '#2ecc71'), (short_call, '#e74c3c'), 
             (long_put, '#3498db'), (short_put, '#f39c12')]

for idx, (payoff, color) in enumerate(positions):
    row, col = divmod(idx, 2)
    fig.add_trace(go.Scatter(
        x=S, y=payoff, fill='tozeroy',
        fillcolor=f'rgba{tuple(int(color[i:i+2], 16) for i in (1, 3, 5)) + (0.3,)}',
        line=dict(color=color, width=2)
    ), row=row+1, col=col+1)
    fig.add_hline(y=0, line_dash='dash', line_color='gray', row=row+1, col=col+1)
    fig.add_vline(x=K, line_dash='dot', line_color='gray', row=row+1, col=col+1)

fig.update_layout(title='Option Payoff Diagrams (K=100)', template='plotly_white', showlegend=False)
fig.update_xaxes(title_text='Stock Price at Expiration')
fig.update_yaxes(title_text='Profit/Loss')
fig

## Black-Scholes Model

The cornerstone of options pricing, derived by Fischer Black and Myron Scholes (1973):

$$C = S_0 N(d_1) - K e^{-rT} N(d_2)$$
$$P = K e^{-rT} N(-d_2) - S_0 N(-d_1)$$

Where:
$$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

**Assumptions:**
- Stock follows geometric Brownian motion
- No dividends, no transaction costs
- Constant volatility and risk-free rate
- European-style options

In [ ]:
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Option price surface
S_range = np.linspace(80, 120, 50)
T_range = np.linspace(0.1, 2, 50)
S_grid, T_grid = np.meshgrid(S_range, T_range)

K, r, sigma = 100, 0.05, 0.2
call_prices = black_scholes(S_grid, K, T_grid, r, sigma, 'call')

fig = go.Figure(data=[go.Surface(x=S_grid, y=T_grid, z=call_prices, colorscale='Viridis')])
fig.update_layout(
    title='Black-Scholes Call Option Price Surface',
    scene=dict(xaxis_title='Stock Price', yaxis_title='Time to Expiry', zaxis_title='Call Price'),
    template='plotly_white'
)
fig

## The Greeks

Sensitivity measures for option positions:

| Greek | Measures | Formula (Call) | Interpretation |
|-------|----------|----------------|----------------|
| **Delta (Δ)** | Price sensitivity | $N(d_1)$ | Change in option price per $1 stock move |
| **Gamma (Γ)** | Delta sensitivity | $\frac{N'(d_1)}{S\sigma\sqrt{T}}$ | Rate of delta change |
| **Theta (Θ)** | Time decay | $-\frac{S N'(d_1)\sigma}{2\sqrt{T}}$ | Daily value erosion |
| **Vega (ν)** | Volatility sensitivity | $S\sqrt{T} N'(d_1)$ | Change per 1% vol move |
| **Rho (ρ)** | Rate sensitivity | $KT e^{-rT} N(d_2)$ | Change per 1% rate move |

In [ ]:
def calculate_greeks(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = -(S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r*T) * norm.cdf(d2)
    vega = S * np.sqrt(T) * norm.pdf(d1) / 100  # per 1% vol change
    
    return delta, gamma, theta/365, vega  # theta per day

S_range = np.linspace(70, 130, 100)
delta, gamma, theta, vega = calculate_greeks(S_range, K=100, T=0.5, r=0.05, sigma=0.2)

fig = make_subplots(rows=2, cols=2, subplot_titles=('Delta', 'Gamma', 'Theta (daily)', 'Vega'))

greeks = [(delta, '#3498db'), (gamma, '#2ecc71'), (theta, '#e74c3c'), (vega, '#9b59b6')]
for idx, (greek, color) in enumerate(greeks):
    row, col = divmod(idx, 2)
    fig.add_trace(go.Scatter(x=S_range, y=greek, line=dict(color=color, width=2)), row=row+1, col=col+1)
    fig.add_vline(x=100, line_dash='dot', line_color='gray', row=row+1, col=col+1)

fig.update_layout(title='Option Greeks vs Stock Price (K=100, T=0.5, σ=20%)', template='plotly_white', showlegend=False)
fig.update_xaxes(title_text='Stock Price')
fig

## Put-Call Parity

A fundamental relationship between European call and put prices:

$$C - P = S_0 - K e^{-rT}$$

This allows traders to:
- Synthetically create positions
- Identify arbitrage opportunities
- Convert between calls and puts

## Common Strategies

| Strategy | Components | Outlook | Max Profit | Max Loss |
|----------|------------|---------|------------|----------|
| **Covered Call** | Long stock + Short call | Neutral/Bullish | Premium + (K-S₀) | S₀ - Premium |
| **Protective Put** | Long stock + Long put | Bullish | Unlimited | S₀ - K + Premium |
| **Straddle** | Long call + Long put | High volatility | Unlimited | Total premium |
| **Iron Condor** | Short OTM strangle + Long further OTM strangle | Low volatility | Net premium | Spread width - Premium |

## Key Takeaways

1. **Options provide leverage** — Control large positions with limited capital
2. **Greeks drive risk management** — Delta hedge to neutralize price risk
3. **Volatility is key** — Vega exposure often dominates option P&L
4. **Time decay is constant** — Theta erodes option value daily
5. **Put-call parity enables arbitrage detection** — Check for mispricings